In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

NumberOfPhases = 3
#Read the labeled training set
#The phases are skyrmions, ferromagnetic and spin spirals
#Labels: skyrmions--0; ferromagnetic--1, spin spirals--2
Skyrmions = pd.read_csv("/users/PAS1495/gsdbuilder/FinalProject/Model/SelectedPhases/Skyrmions_z.csv")
Ferromagnetics = pd.read_csv("/users/PAS1495/gsdbuilder/FinalProject/Model/SelectedPhases/Ferrogmanetic_z.csv")
Spirals = pd.read_csv("/users/PAS1495/gsdbuilder/FinalProject/Model/SelectedPhases/Spiral_z.csv")

#For a file will all the data, xspin is 0~783, yspin is 784~1567, zspin is 1568~2351,
#Correspondingly, iloc[:, 0:784], iloc[:, 784:1568], iloc[:, 1568:2352]
#For now, we will use the zspin first. A smaller file is used and the zspin is now 0~783.
file = pd.DataFrame()
file = pd.concat([file, Skyrmions])
file = pd.concat([file, Ferromagnetics])
file = pd.concat([file, Spirals])

#Add the average zspin to the end of the array.
data_dimz = file.iloc[:, 0:784].values
data_ave_zspin = file.iloc[:, 788].values.reshape(-1, 1)
print(data_dimz.shape, data_ave_zspin.shape)
data_dimz_avez = np.append(data_dimz, data_ave_zspin, axis=1)

#The label of the data is the column #2356
data_labels = file.iloc[:, 789].values
data_labels_onehot = np.zeros((len(data_labels), NumberOfPhases))
for i in range(len(data_labels)):
    data_labels_onehot[i][data_labels[i]] = 1
    
size = len(data_dimz)
print("The total size of dataset is ", size)
print("We will now select random 80% of the data as train data, and the rest serves as test data")

(4395, 784) (4395, 1)
The total size of dataset is  4395
We will now select random 80% of the data as train data, and the rest serves as test data


In [3]:
#Shuffle the data before we choose our train/test dataset.
new_order = np.random.permutation(range(size))
train_range = new_order[0:int(0.8*size)] #The training sample is 80% of the total dataset.
test_range = new_order[int(0.8*size):size]

NumberOfPhases = 3
x_train = data_dimz[train_range]
y_train_onehot = data_labels_onehot[train_range]

x_test = data_dimz[test_range]
y_test = data_labels_onehot[test_range]

print("The training and testing sample sizes are ")
print(len(train_range), len(test_range))

The training and testing sample sizes are 
3516 879


In [4]:
from keras import models
from keras import layers
from keras.models import load_model

#Load the trained encoder model
encoder = load_model('fully_trained_encoder.h5')
for layer in encoder.layers:
    layer.trainable = False
    print(layer, "trainable? ", layer.trainable)
    
print(encoder.predict(x_train)[1])

<keras.engine.input_layer.InputLayer object at 0x2b5847f57588> trainable?  False
<keras.layers.core.Dense object at 0x2b5847f575f8> trainable?  False
<keras.layers.core.Dense object at 0x2b5847f577b8> trainable?  False
<keras.layers.core.Dense object at 0x2b5847f57898> trainable?  False
<keras.layers.core.Lambda object at 0x2b5847f57b38> trainable?  False
[[-0.56097114 -0.45866722 -0.6229448  ... -0.40006357 -0.38145673
  -0.4939673 ]
 [-0.5550833  -0.46567804 -0.62535137 ... -0.41345727 -0.37177938
  -0.4884244 ]
 [-0.5629843  -0.4667532  -0.63682044 ... -0.41286734 -0.38129795
  -0.5018695 ]
 ...
 [-3.6175203  -3.515184   -3.2352068  ... -2.591853   -2.4889874
  -3.0969837 ]
 [-2.1503935  -2.1226776  -1.9930882  ... -1.8141134  -1.9406321
  -2.0287836 ]
 [-2.6645608  -2.1888058  -2.710416   ... -2.4153073  -2.215819
  -2.4635484 ]]


/users/PAS1495/gsdbuilder/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras import metrics

latent_dim = 100
intermediate_dim = 64

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
#
# This is an intermediate layer for the decoder
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
#
# The output layer has sigmoid activation (just like our normal autoencoder)
# and the same dimension as our input
outputs = Dense(NumberOfPhases, activation='sigmoid')(x)

# instantiate decoder model
fcn = Model(latent_inputs, outputs, name='decoder')
fcn.summary()
plot_model(fcn, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
# ==> This next line defines the output as the output of the decoder (of course)
# ==> The input to the decoder is the *third* output of the encoder ==> the z vector
#The mean and variance output is not used here.
outputs = fcn(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

# Now compile our model!
vae.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("summary")
print(vae.summary())
plot_model(vae, to_file='vae_mlp.png', show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 6,659
Trainable params: 6,659
Non-trainable params: 0
_________________________________________________________________


NameError: name 'inputs' is not defined

In [ ]:
epochs = 5
batch_size = 256
history = stacked_network.fit(x_train, y_train_onehot, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test_onehot))